In [3]:

import pandas as pd
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.ensemble import VotingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import QuantileTransformer

In [4]:
X, y = fetch_california_housing(return_X_y=True)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
models = [
    ['tree', DecisionTreeRegressor()],
    ['knn', KNeighborsRegressor()]
]

voter = VotingRegressor(models)

In [7]:
steps = [
    [ 'scaler', QuantileTransformer() ],
    [ 'voter', voter]
]

pipe = Pipeline(steps)

In [8]:
params = {
    'voter__tree__max_depth'  : [10, 100, 1000],
    'voter__knn__n_neighbors' : [2, 10, 100]
}

grid = GridSearchCV(pipe, param_grid=params, cv=2)

In [9]:
grid.fit(X_train, y_train)

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[['scaler', QuantileTransformer()],
                                       ['voter',
                                        VotingRegressor(estimators=[['tree',
                                                                     DecisionTreeRegressor()],
                                                                    ['knn',
                                                                     KNeighborsRegressor()]])]]),
             param_grid={'voter__knn__n_neighbors': [2, 10, 100],
                         'voter__tree__max_depth': [10, 100, 1000]})

In [10]:
results = pd.DataFrame(grid.cv_results_)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_voter__knn__n_neighbors,param_voter__tree__max_depth,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,0.098233,0.010440,0.225434,0.008052,2,10,"{'voter__knn__n_neighbors': 2, 'voter__tree__m...",0.723573,0.717356,0.720465,0.003108,4
1,0.119216,0.007480,0.228866,0.006463,2,100,"{'voter__knn__n_neighbors': 2, 'voter__tree__m...",0.710346,0.700704,0.705525,0.004821,7
2,0.109694,0.001989,0.223407,0.000974,2,1000,"{'voter__knn__n_neighbors': 2, 'voter__tree__m...",0.710239,0.701543,0.705891,0.004348,6
3,0.100235,0.010472,0.389455,0.000498,10,10,"{'voter__knn__n_neighbors': 10, 'voter__tree__...",0.739115,0.736573,0.737844,0.001271,1
4,0.128158,0.005485,0.445309,0.015459,10,100,"{'voter__knn__n_neighbors': 10, 'voter__tree__...",0.732494,0.724636,0.728565,0.003929,2
5,0.148106,0.001493,0.399429,0.029423,10,1000,"{'voter__knn__n_neighbors': 10, 'voter__tree__...",0.731967,0.720501,0.726234,0.005733,3
6,0.096757,0.001020,1.068623,0.028405,100,10,"{'voter__knn__n_neighbors': 100, 'voter__tree_...",0.713158,0.713674,0.713416,0.000258,5
7,0.140606,0.009989,1.032738,0.004488,100,100,"{'voter__knn__n_neighbors': 100, 'voter__tree_...",0.707579,0.695383,0.701481,0.006098,9
8,0.115193,0.003492,1.041713,0.020448,100,1000,"{'voter__knn__n_neighbors': 100, 'voter__tree_...",0.707017,0.698742,0.702879,0.004137,8


In [11]:
grid.best_estimator_

Pipeline(steps=[('scaler', QuantileTransformer()),
                ['voter',
                 VotingRegressor(estimators=[['tree',
                                              DecisionTreeRegressor(max_depth=10)],
                                             ['knn',
                                              KNeighborsRegressor(n_neighbors=10)]])]])

In [12]:
grid.predict(X_test[:2])

array([1.8633    , 2.65825329])